<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#JTBC-토요일-23시-방영작-크롤링" data-toc-modified-id="JTBC-토요일-23시-방영작-크롤링-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>JTBC 토요일 23시 방영작 크롤링</a></span><ul class="toc-item"><li><span><a href="#URL-분석" data-toc-modified-id="URL-분석-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>URL 분석</a></span></li><li><span><a href="#크롤링-전략" data-toc-modified-id="크롤링-전략-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>크롤링 전략</a></span></li></ul></li><li><span><a href="#시청률-크롤링" data-toc-modified-id="시청률-크롤링-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>시청률 크롤링</a></span><ul class="toc-item"><li><span><a href="#URL-분석" data-toc-modified-id="URL-분석-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>URL 분석</a></span></li><li><span><a href="#크롤링-전략" data-toc-modified-id="크롤링-전략-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>크롤링 전략</a></span></li></ul></li><li><span><a href="#트위터-언급량-크롤링" data-toc-modified-id="트위터-언급량-크롤링-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>트위터 언급량 크롤링</a></span></li></ul></div>

In [23]:
# libraries
from selenium.webdriver.support.ui import Select
from selenium import webdriver

import numpy as np  # linear algebra
import pandas as pd  # file I/O
import requests
import json
import bs4

import re  # regex
import time  # time sleep

from twitterscraper import query_tweets  # 트위터 크롤러

In [3]:
driver = webdriver.Chrome('D:\document\chrome_driver\chromedriver')  # start webdriver

## JTBC 토요일 23시 방영작 크롤링
### URL 분석
* JTBC 편성표 url은 /schedule/날짜 로 구성됐다.
* 날짜는 'yyyymmdd' 형식이다.

### 크롤링 전략
* 2018010 - 20190720 에서 토요일에 해당되는 날짜 리스트 생성
* 모든 토요일 날짜 리스트에 대하여
    * url = 'http://jtbc.joins.com/schedule/yyyymmdd' 선언 후 접속
    * 가장 마지막 li(23시) 선택
        * date(날짜)
        * time(방영시간)
        * title(방송명)
    * 저장

In [4]:
# 20180801 - 20190720 토요일 날짜 리스트 생성
date_idx = pd.date_range(start='20180101', end='20190720', freq='W-SAT')
date_array = date_idx.values  # to np array
print(f'raw date: {date_sat[0]}')

# yyyy-mm-ddThh:mm:ss.ssssssss=> yyymmdd로 변환
date_transformed_list = []  # yyymmdd
for date in date_array:
    date_str = str(date)  # np.datetime64 to str
    date_yyymmdd = date_str.split('T')[0]  # T를 기준으로 분할
    date_transformed = re.sub('-','',date_yyymmdd)  # - 제거
    date_transformed_list.append(date_transformed)  # 리스트에 저장
print(f'date transformed: {date_transformed_list[0]}')        

NameError: name 'date_sat' is not defined

In [ ]:
jtbc_programs = {
    'date': [],
    'time': [],
    'title': []
}

for date in date_transformed_list:
    # 접속
    url = f'http://jtbc.joins.com/schedule/{date}'
    driver.get(url)

    # get ul last child
    program_info = driver.find_element_by_css_selector('ul.chart_time_list > li:last-child').text.split('\n')

    # save date to dict
    jtbc_programs['date'].append(date)
    jtbc_programs['time'].append(program_info[0].strip())
    jtbc_programs['title'].append(program_info[1].strip())
    
    time.sleep(1)

In [75]:
# save as csv
jtbc_programs_df = pd.DataFrame.from_dict(jtbc_programs)
jtbc_programs_df.to_csv('jtbc_programs.csv', index=False)

## 시청률 크롤링
* 닐슨코리아 시청률, 시정차수 순위(TOP10)에서 JTBC 토요일 23시 방영작 시청률

### URL 분석
* url = 'https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=2_1&area=00&begin_date=날짜' 로구성됐다.
* area
    * 00: 전국
    * 01: 수도권
* 날짜는 yyyymmdd 형식

### 크롤링 전략
* 20180101 - 20190720 의 토요일에 해당하는 날짜 생성
* 모든 토요일에 해당하는 날짜에 대하여
    * url = 'https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=2_1&area=01&begin_date=날짜' 선언 및 접속
    * 시청률
        * date, channel, ranking, title, rating
    * 시청자 수
        * date, channel, ranking, title, views
    * 저장

In [257]:
rating_rank = {
    'date': [],
    'rank': [],
    'title': [],
    'channel': [],
    'rating': []
}

In [258]:
i = 0
for date in date_transformed_list:
    i += 1
    # 접속
    url = f'https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=2_1&area=00&begin_date={date}'
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'lxml', from_encoding='utf-8')
    table_rank = soup.select('table.ranking_tb>tr')  # get rating table
    
    # 3-13 번째 row(top10)
    for idx in range(3, 13):
        current_row = table_rank[idx]
        tmp = re.sub('\t', '', current_row.text).strip().split('\n')
        rating_rank['date'].append(date)
        rating_rank['rank'].append(tmp[0])
        rating_rank['channel'].append(tmp[1])
        rating_rank['title'].append(tmp[2])
        rating_rank['rating'].append(tmp[4])
    print(f'{date}: "{i}/{len(date_transformed_list)}')
    time.sleep(1)

20180106: "1/81
20180113: "2/81
20180120: "3/81
20180127: "4/81
20180203: "5/81
20180210: "6/81
20180217: "7/81
20180224: "8/81
20180303: "9/81
20180310: "10/81
20180317: "11/81
20180324: "12/81
20180331: "13/81
20180407: "14/81
20180414: "15/81
20180421: "16/81
20180428: "17/81
20180505: "18/81
20180512: "19/81
20180519: "20/81
20180526: "21/81
20180602: "22/81
20180609: "23/81
20180616: "24/81
20180623: "25/81
20180630: "26/81
20180707: "27/81
20180714: "28/81
20180721: "29/81
20180728: "30/81
20180804: "31/81
20180811: "32/81
20180818: "33/81
20180825: "34/81
20180901: "35/81
20180908: "36/81
20180915: "37/81
20180922: "38/81
20180929: "39/81
20181006: "40/81
20181013: "41/81
20181020: "42/81
20181027: "43/81
20181103: "44/81
20181110: "45/81
20181117: "46/81
20181124: "47/81
20181201: "48/81
20181208: "49/81
20181215: "50/81
20181222: "51/81
20181229: "52/81
20190105: "53/81
20190112: "54/81
20190119: "55/81
20190126: "56/81
20190202: "57/81
20190209: "58/81
20190216: "59/81
201902

In [259]:
len(date_transformed_list)

81

In [260]:
len(rating_rank['date'])

810

In [261]:
# save as csv
rating_rank_df = pd.DataFrame.from_dict(rating_rank)
rating_rank_df.to_csv('rating_rank.csv', index=False)

In [173]:
def requestPage(url):
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'lxml', from_encoding='utf-8')
    return soup

In [242]:
len(date_transformed_list)

810

## 트위터 언급량 크롤링
* 본문, 계정에 방탄, bts, BTS를 포함한 트위터 게시물 수 카운트
* reference: [twitterscraper](https://github.com/taspinar/twitterscraper/blob/master/twitterscraper/query.py)

In [38]:
twit_list = query_tweets('방탄 OR BTS OR bts', begindate=datetime.date(2019,6,15), enddate=datetime.date(2019,7,28), poolsize=150, lang='ko')

In [39]:
twit_dict = {
    'id':[],
    'date':[],
    'content':[]
}

for twit in tweet_list:
    twit_dict['id'].append(twit.fullname)
    twit_dict['date'].append(twit.timestamp)
    twit_dict['content'].append(twit.text)

# save to csv
pd.DataFrame.from_dict(twit_dict).to_csv('bangtan_twitter.csv', index=False)